In [33]:
from pathlib import Path
import nrrd
import numpy as np

In [74]:
imgs = Path("imagesTr")
labs = Path("labelsTr")


def get_seqs(datapath):

    segmentation_files = []
    sequence_files = []

    segpath = datapath/labs
    seqpath = datapath/imgs

    segmentation_files = [file for file in segpath.glob("*.nrrd")]
    sequence_files = [file for file in seqpath.glob("*.nrrd")]

    segmentation_files.sort(), sequence_files.sort()

    return sequence_files, segmentation_files

def transpose(seqs, segs):
    new_seq = []
    new_seg = []
    for seq in seqs:
        data, header = nrrd.read(seq)
        if data.shape[0] > data.shape[1]:
            new_seq.append(seq)
            nrrd.write(str(seq), data.transpose((1, 0, 2)))
        elif data.shape[0] < data.shape[1]:
            new_seq.append(seq)
        elif data.shape[0] == data.shape[1]:
            new_seq.append(seq)
        else:
            print("Something wrong")

    for seg in segs:
        data, header = nrrd.read(seq)
        if data.shape[0] > data.shape[1]:
            new_seg.append(seg)
            nrrd.write(str(seg), data.transpose((1, 0, 2)))
        elif data.shape[0] < data.shape[1]:
            new_seg.append(seg)
        elif data.shape[0] == data.shape[1]:
            new_seg.append(seg)
        else:
            print("Something wrong")
    return new_seq, new_seg


def crop_to_size(seqs, segs, x, y):
    return seqs[:,0:x,0:y,:], segs[:,0:x,0:y,:]

def pad_with_zeros(seqs, segs, pad_dim=(128, 128, 31)):
    new_seq = []
    new_seg = []

    for seq in seqs:
        data_seq, _ = nrrd.read(seq)
        pad = np.zeros(pad_dim)
        pad[0:data_seq.shape[0], 0:data_seq.shape[1], 0:data_seq.shape[2]] = data_seq
        nrrd.write(str(seq), pad)
        new_seq.append(seq)
    
    for seg in segs:
        data_seg, _ = nrrd.read(seg)
        pad = np.zeros(pad_dim)
        pad[0:data_seg.shape[0], 0:data_seg.shape[1], 0:data_seg.shape[2]] = data_seg
        nrrd.write(str(seg), pad)
        new_seg.append(seg)

    return new_seq, new_seg

def allow_only_by(seqs, segs, size=(128, 256, 20)):
    new_seqs = []
    new_segs = []

    for seq, seg in zip(seqs, segs):
        data_seq, _ = nrrd.read(seq)
        data_seg, _ = nrrd.read(seg)
        if data_seq.shape == size and data_seg.shape == size:
            new_seqs.append(seq)
            new_segs.append(seg)

    return new_seqs, new_segs



In [100]:
unet_path_1 = Path("../nnUNet_raw/Dataset001_T1/")
unet_path_2 = Path("../nnUNet_raw/Dataset002_T2/")
unet_path_3 = Path("../nnUNet_raw/Dataset003_T2star/")
unet_path_4 = Path("../nnUNet_raw/Dataset004_ASL/")
unet_path_5 = Path("../nnUNet_raw/Dataset005_DWI/")

seq_names = ["T1", "T2", "T2star", "ASL", "DWI"]
paths = [unet_path_1, unet_path_2, unet_path_3, unet_path_4, unet_path_5]

# T2

In [35]:
seqs, segs = get_seqs(unet_path_2)
seqs, segs = transpose(seqs, segs)

In [59]:
for seq, seg in zip(seqs, segs):
    name_seq = seq.name
    clean_out_seq = seq.parent/"clean"/name_seq
    seq.rename(clean_out_seq)

    name_seg = seg.name
    clean_out_seg = seg.parent/"clean"/name_seg
    seg.rename(clean_out_seg)

# T2star

In [71]:
seqs, segs = get_seqs(unet_path_3)
seqs, segs = transpose(seqs, segs)
seqs, segs = allow_only_by(seqs, segs)

for seq, seg in zip(seqs, segs):
    name_seq = seq.name
    clean_out_seq = seq.parent/"clean"/name_seq
    seq.rename(clean_out_seq)

    name_seg = seg.name
    clean_out_seg = seg.parent/"clean"/name_seg
    seg.rename(clean_out_seg)

# ASL

In [78]:
seqs, segs = get_seqs(unet_path_4)
seqs, segs = pad_with_zeros(seqs, segs)

for seq, seg in zip(seqs, segs):
    name_seq = seq.name
    clean_out_seq = seq.parent/"clean"/name_seq
    seq.rename(clean_out_seq)

    name_seg = seg.name
    clean_out_seg = seg.parent/"clean"/name_seg
    seg.rename(clean_out_seg)

# DWI

In [79]:
seqs, segs = get_seqs(unet_path_5)
seqs, segs = transpose(seqs, segs)
seqs, segs = allow_only_by(seqs, segs, size=(104, 128, 15))


for seq, seg in zip(seqs, segs):
    name_seq = seq.name
    clean_out_seq = seq.parent/"clean"/name_seq
    seq.rename(clean_out_seq)

    name_seg = seg.name
    clean_out_seg = seg.parent/"clean"/name_seg
    seg.rename(clean_out_seg)

## Format to the right name

In [ ]:
for (path, sequence_name) in zip(paths, seq_names):
    seqs, segs = get_seqs(path)
    print(path, sequence_name)
    for i, (seq, seg) in enumerate(zip(seqs, segs)):
        if seq.name == seg.name:
            seq_name = f"{sequence_name}_{i:03d}_0000.nrrd"
            seg_name = f"{sequence_name}_{i:03d}.nrrd"

            # rename seq
            seq_ori = seq.name
            clean_out_seq = seq.parent/seq_name
            seq.rename(clean_out_seq)

            # rename seg
            seg_ori = seg.name
            clean_out_seg = seg.parent/seg_name
            seg.rename(clean_out_seg)